In [57]:
from pyDigitalWaveTools.vcd.parser import VcdParser
import pandas as pd

def load_signals_to_df(root, names):
    data = [root.children[name].data for name in names]

    max_time = max([max([kv[0] for kv in d]) for d in data])

    df = None
    for i in range(len(data)):
        name = names[i]
        d = data[i]

        tmp = pd.DataFrame(d, columns=['time', name]).set_index(['time'])
        tmp = tmp.reindex(pd.RangeIndex(0, max_time+1), method='ffill')

        if i == 0:
            df = tmp
        else:
            df = df.join(tmp)

    return df

def load_trace(filename):
    with open(filename) as fp:
        vcd = VcdParser()
        vcd.parse(fp)

        root = vcd.scope.children['TOP'] \
                        .children['ibex_demo_system'] \
                        .children['u_top'] \
                        .children['u_ibex_core'] \
                        .children['if_stage_i'] \
                        .children['gen_prefetch_buffer'] \
                        .children['prefetch_buffer_i']

        return load_signals_to_df(root, ['addr_o[31:0]', 'rdata_o[31:0]', 'ready_i', 'valid_o', 'branch_i'])

In [58]:
ibex_trace_df = load_trace('../sim_ibex.vcd')
shufflev_trace_df = load_trace('../sim_shufflev.vcd')

ibex_trace_df_filter = ibex_trace_df[(ibex_trace_df['ready_i'] == '1') & (ibex_trace_df['valid_o'] == '1') & (ibex_trace_df['branch_i'] == '0')]
shufflev_trace_df_filter = shufflev_trace_df[(shufflev_trace_df['ready_i'] == '1') & (shufflev_trace_df['valid_o'] == '1') & (shufflev_trace_df['branch_i'] == '0')]

In [60]:
for t1, t2 in zip(ibex_trace_df_filter.iterrows(), shufflev_trace_df_filter.iterrows()):
    # print (t1[0], hex(int(t1[1]['addr_o[31:0]'][1:], base=2)), hex(int(t1[1]['rdata_o[31:0]'][1:], base=2)), 
    #        t2[0], hex(int(t2[1]['addr_o[31:0]'][1:], base=2)), hex(int(t2[1]['rdata_o[31:0]'][1:], base=2)))
    if (t1[1]['addr_o[31:0]'] != t2[1]['addr_o[31:0]']) or (t1[1]['rdata_o[31:0]'] != t2[1]['rdata_o[31:0]']):
        print ('Trace mismatch at timestamp: ', t1[0], t2[0])
        break